# **Modelo Temporal Previsão de Risco de Diabetes**

## **Importações**

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

## **Carregamento e preparação dos dados**

In [10]:
import pandas as pd

#Lê o arquivo CSV
df = pd.read_csv(r'https://raw.githubusercontent.com/vitoriall/diabetes-prediction-raitec/refs/heads/main/data/dados_pacientes_diabetes%20temporal.csv')

#Converte a coluna 'data' para tipo datetime
df['data'] = pd.to_datetime(df['data'])

# Ordenar os dados por paciente e data
df = df.sort_values(by=['paciente_id', 'data'])

## **Criação de variáveis temporais (lags, médias móveis, diferenças)**

In [11]:
def add_temporal_features(group):
    #Copia o grupo para evitar avisos do pandas sobre alteração de dados em slices da tabela original.
    group = group.copy()

    #shift(n) pega o valor de glicemia de n dias atrás.
    #Isso permite capturar como a glicemia estava antes, e identificar aumentos ou quedas.
    group['glicemia_lag1'] = group['glicemia'].shift(1)
    group['glicemia_lag3'] = group['glicemia'].shift(3)
    group['glicemia_lag7'] = group['glicemia'].shift(7)

    #Isso calcula a média da glicemia nos últimos 3, 5 ou 7 registros.
    #Serve para suavizar a série e detectar tendências de forma mais robusta.
    group['glicemia_ma3'] = group['glicemia'].rolling(window=3).mean()
    group['glicemia_ma5'] = group['glicemia'].rolling(window=5).mean()
    group['glicemia_ma7'] = group['glicemia'].rolling(window=7).mean()

    # Captura a variação da glicemia entre hoje e 1 ou 3 dias atrás.
    group['glicemia_diff1'] = group['glicemia'] - group['glicemia_lag1']
    group['glicemia_diff3'] = group['glicemia'] - group['glicemia_lag3']

    return group
#Agrupa os dados por paciente_id, e aplica a função a cada paciente separadamente.
df = df.groupby('paciente_id').apply(add_temporal_features)

#Remoção de valores faltantes gerados
df = df.dropna(subset=['glicemia_lag1', 'glicemia_ma3', 'glicemia_ma7', 'glicemia_diff1'])

# Alvo binário
df['alerta_risco'] = ((df['glicemia'] > df['glicemia_ma3'] * 1.1) | (df['glicemia_diff1'] > 20)).astype(int)


<ipython-input-11-782077433>:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('paciente_id').apply(add_temporal_features)


## **Criação de variáveis preditoras + Divisão em treino e teste**

In [12]:
#lista com os nomes das colunas (features) que serão usadas como entradas para o modelo.
features = ['idade', 'imc', 'glicemia', 'glicemia_lag1', 'glicemia_ma3', 'glicemia_ma7']

#X é o DataFrame que contém todas as linhas e apenas as colunas que definimos como features
X = df[features]

#y é a série (coluna) do dataframe que contém o alvo (label), que nesse caso é a coluna alerta_risco
y = df['alerta_risco']

# Divisão temporal (80% mais antigas para treino)
cutoff_date = df['data'].quantile(0.8)
train_df = df[df['data'] <= cutoff_date]
test_df = df[df['data'] > cutoff_date]

#Aqui você será separado os dados de treino e teste em variáveis para as features
X_train = train_df[features]
y_train = train_df['alerta_risco']
X_test = test_df[features]
y_test = test_df['alerta_risco']

## **Padronização + treinamento e predição**

In [13]:
scaler = StandardScaler()
# Padronização dos dados
#Isso não muda a relação entre os dados, mas garante que todas as variáveis estejam na mesma escala, o que é essencial para muitos algoritmos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Conversão para DMatrix
#DMatrix é uma estrutura de dados otimizada para o XGBoost. Ela melhora a eficiência na memória e na velocidade durante o treinamento e predição
dtrain = xgb.DMatrix(X_train_scaled, label=y_train, feature_names=features)
dtest = xgb.DMatrix(X_test_scaled, label=y_test, feature_names=features)

# Define os parâmetros para o treinamento do XGBoost
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 4,
    'eta': 0.1,
    'seed': 42
}
model = xgb.train(params, dtrain, num_boost_round=100)

# model.predict() retorna as probabilidades estimadas de a classe ser 1 (alerta de risco)
y_train_pred = (model.predict(dtrain) > 0.5).astype(int)
y_test_pred = (model.predict(dtest) > 0.5).astype(int)

## **Avaliação do modelo**

In [14]:
# Avaliação
print("==== DESEMPENHO NO TREINO ====")
print(classification_report(y_train, y_train_pred))

print("==== DESEMPENHO NO TESTE ====")
print(classification_report(y_test, y_test_pred))

==== DESEMPENHO NO TREINO ====
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     22887
           1       0.99      0.96      0.98      5913

    accuracy                           0.99     28800
   macro avg       0.99      0.98      0.99     28800
weighted avg       0.99      0.99      0.99     28800

==== DESEMPENHO NO TESTE ====
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5650
           1       0.97      0.95      0.96      1450

    accuracy                           0.98      7100
   macro avg       0.98      0.97      0.98      7100
weighted avg       0.98      0.98      0.98      7100



## **Gerar alertas previstos com base nas previsões do modelo**

In [15]:
#Aqui é criado uma cópia do DataFrame de teste para evitar modificar o original.
test_df = test_df.copy()

#Adiciona uma nova coluna chamada 'alerta_previsto' no DataFrame de teste.
test_df['alerta_previsto'] = y_test_pred

#Filtra o DataFrame para pegar apenas as linhas onde o modelo previu alerta (valor 1).
alertas = test_df[test_df['alerta_previsto'] == 1][['data', 'paciente_id', 'glicemia']]
print("\nExemplos de alertas previstos:")
print(alertas.head())


Exemplos de alertas previstos:
                        data  paciente_id  glicemia
paciente_id                                        
1           30200 2023-10-30            1     112.2
            30500 2023-11-02            1     126.9
            31200 2023-11-09            1     128.0
            31400 2023-11-11            1     103.2
            31500 2023-11-12            1     122.5
